<a href="https://colab.research.google.com/github/DavoodSZ1993/Dive_into_Deep_Learning/blob/main/6_builder_guide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 6.1 Layers and Modules

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F

In [ ]:
net = nn.Sequential(nn.LazyLinear(256),   # input x 256
                    nn.ReLU(),            
                    nn.LazyLinear(10))    # 256 x 10

X = torch.rand(2, 20) # two samples (rows) & 20 features (columns)
X.shape, net(X).shape

(torch.Size([2, 20]), torch.Size([2, 10]))

### 6.1.1 A Custom Module

In [ ]:
class MLP(nn.Module):
  def __init__(self):
    super().__init__()               # Performs necessary initialization from the parent class.
    self.hidden = nn.LazyLinear(256)
    self.out = nn.LazyLinear(10)

  def forward(self, X):              # Does the forward propagation for the given network.
    return self.out(F.relu(self.hidden(X)))

In [ ]:
net = MLP()
net(X).shape

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


torch.Size([2, 10])

### 6.1.2 The Sequential Module

In [ ]:
class MySequential(nn.Module):
  def __init__(self, *args):
    super().__init__()
    for idx, module in enumerate(args):
      self.add_module(str(idx), module)

  def forward(self, X):
    for module in self.children():
      X = module(X)
    return X

In [ ]:
net_layers = (nn.LazyLinear(256),
              nn.ReLU(),
              nn.LazyLinear(10))
net = MySequential(*net_layers)

net(X).shape

torch.Size([2, 10])

### 6.1.3 Executing Code in the Forward Propagation Method

In [ ]:
class FixedHiddenMLP(nn.Module):
  def __init__(self):
    super().__init__()

    self.rand_weight = torch.rand((20, 20))
    self.linear = nn.LazyLinear(20)

  def forward(self, X):
   
    X = self.linear(X)
    X = F.relu(X @ self.rand_weight + 1)
    X = self.linear(X)

    # Control flow
    while X.abs().sum() > 1:
      X /=2

    return X.sum()

In [ ]:
net = FixedHiddenMLP()
net(X)

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


tensor(0.0227, grad_fn=<SumBackward0>)

In [ ]:
class NestMLP(nn.Module):
  def __init__(self):
    super().__init__()

    self.net = nn.Sequential(nn.LazyLinear(64),
                             nn.ReLU(),
                             nn.LazyLinear(32),
                             nn.ReLU())
    self.linear = nn.LazyLinear(16)

  def forward(self, X):
    return self.linear(self.net(X))

In [ ]:
chimera = nn.Sequential(NestMLP(),
                        nn.LazyLinear(20),
                        FixedHiddenMLP())

chimera(X)

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


tensor(0.0920, grad_fn=<SumBackward0>)

## 6.2 Parameter Management

In [ ]:
import torch
from torch import nn

X = torch.rand(size=(2, 4))  # two inputs, four features

In [ ]:
net = nn.Sequential(nn.LazyLinear(8),
                    nn.ReLU(),
                    nn.LazyLinear(1))

net(X).shape

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


torch.Size([2, 1])

### 6.2.1 Parameter Access

In [ ]:
net[2].state_dict()

OrderedDict([('weight',
              tensor([[ 0.1928, -0.2604,  0.2596,  0.0229,  0.0822, -0.0394, -0.0623, -0.0491]])),
             ('bias', tensor([0.0160]))])

#### Targeted Parameters

In [ ]:
type(net[2].bias), net[2].bias.data

(torch.nn.parameter.Parameter, tensor([0.0160]))

In [ ]:
net[2].weight.grad == None

True

#### All Parameters at Once

In [ ]:
[(name, param.shape) for name, param in net.named_parameters()]

[('0.weight', torch.Size([8, 4])),
 ('0.bias', torch.Size([8])),
 ('2.weight', torch.Size([1, 8])),
 ('2.bias', torch.Size([1]))]

### 6.2.2 Tied Parameters


In [ ]:
shared = nn.LazyLinear(8)
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.LazyLinear(1))

net(X)

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


tensor([[-0.3695],
        [-0.3639]], grad_fn=<AddmmBackward0>)

In [ ]:
print(net[2].weight.data[0] == net[4].weight.data[0])

net[2].weight.data[0, 0] = 100                        # Changes the weight in both layers.
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


## 6.3 Parameter Initialization

In [ ]:
import torch
from torch import nn

X = torch.rand(size=(2, 4))

In [ ]:
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(),
                    nn.LazyLinear(1))

net(X).shape

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


torch.Size([2, 1])

### 6.3.1 Built-in Initialization

In [ ]:
def init_normal(module):
  if type(module) == nn.Linear:
    nn.init.normal_(module.weight, mean=0, std=0.01)
    nn.init.zeros_(module.bias)

In [ ]:
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0012, -0.0020, -0.0085,  0.0142]), tensor(0.))

In [ ]:
def init_constant(module):
  if type(module) == nn.Linear:
    nn.init.constant_(module.weight, 1)
    nn.init.zeros_(module.bias)

In [ ]:
net.apply(init_constant)

net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [ ]:
def init_xavier(module):
  if type(module) == nn.Linear:
    nn.init.xavier_uniform_(module.weight)

def init_42(module):
  if type(module) == nn.Linear:
    nn.init.constant_(module.weight, 42)

In [ ]:
net[0].apply(init_xavier)
net[2].apply(init_42)

print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.1237,  0.1062, -0.0510, -0.5620])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


#### Custom Initialization

In [ ]:
def my_init(module):
  if type(module) == nn.Linear:
    print("Init", *[(name, param.shape)
                      for name, param in module.named_parameters()][0])
    nn.init.uniform_(module.weight, -10, 10)
    module.weight.data *= module.weight.data >= 5

In [ ]:
net.apply(my_init)

net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[9.7712, -0.0000, -0.0000, 7.0794],
        [5.1773, 5.2045, 7.1202, 0.0000]], grad_fn=<SliceBackward0>)

In [ ]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  1.0000,  1.0000,  8.0794])

## 6.4 Lazy Initialization

In [1]:
!pip install d2l==1.0.0-alpha1.post0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.0/93.0 KB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 KB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 KB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.1 MB/s eta 0:00:00


In [2]:
import torch
from torch import nn
from d2l import torch as d2l

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [3]:
net = nn.Sequential(nn.LazyLinear(256),
                    nn.ReLU(),
                    nn.LazyLinear(10))

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [4]:
net[0].weight

<UninitializedParameter>

In [5]:
X = torch.rand(size=(2, 20))
net(X)
net[0].weight.shape

torch.Size([256, 20])

In [6]:
@d2l.add_to_class(d2l.Module)
def apply_init(self, inputs, init=None):
  self.forward(*inputs)
  if init is not None:
    self.net.apply(init)

## 6.5 Custom Layers

### 6.5.1 Layers without Parameters

In [7]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

In [8]:
class CenteredLayer(nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self, X):
    return X - X.mean()

In [9]:
layer = CenteredLayer()

X = torch.tensor([1.0, 2, 3, 4, 5])
layer(X)

tensor([-2., -1.,  0.,  1.,  2.])

In [10]:
net = nn.Sequential(nn.LazyLinear(128),
                    CenteredLayer())


/usr/local/lib/python3.8/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [11]:
Y = net(torch.rand(size=(4, 8)))
Y.mean()                          # Should be zero!!

tensor(4.1910e-09, grad_fn=<MeanBackward0>)

### 6.5.2 Layers with Parameters

In [13]:
class MyLinear(nn.Module):
  def __init__(self, in_units, units):
    super().__init__()

    self.weight = nn.Parameter(torch.randn(in_units, units))
    self.bias = nn.Parameter(torch.randn(units,))

  def forward(self, X):
    linear = torch.matmul(X, self.weight.data) + self.bias.data
    return F.relu(linear)


In [14]:
linear = MyLinear(5, 3)
linear.weight, linear.bias

(Parameter containing:
 tensor([[ 0.5356,  2.5142,  0.2034],
         [ 0.3318,  0.2023,  1.5738],
         [-2.0756,  1.6652, -0.7115],
         [-0.2573, -0.8007, -0.8965],
         [ 0.5255,  1.3862,  0.9506]], requires_grad=True),
 Parameter containing:
 tensor([0.7425, 1.4366, 0.3611], requires_grad=True))

In [15]:
linear(torch.rand(2, 5))

tensor([[0.0403, 3.6304, 0.9692],
        [0.0000, 4.2660, 0.8591]])

In [16]:
net = nn.Sequential(MyLinear(64, 8),
                    MyLinear(8, 1))

net(torch.rand(2, 64))

tensor([[0.],
        [0.]])

## 6.6 File I/O

### 6.6.1 Loading and Saving Tensors

In [17]:
import torch
from torch import nn
from torch.nn import functional as F

In [18]:
x = torch.arange(4)
torch.save(x, 'x-file')

In [20]:
x2 = torch.load('x-file')
x2 == x

tensor([True, True, True, True])

In [21]:
y = torch.zeros(4)
torch.save([x, y], 'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [22]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')

mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

### 6.6.2 Loading and Saving Model Parameters

In [23]:
class MLP(nn.Module):
  def __init__(self):
    super().__init__()

    self.hidden = nn.LazyLinear(256)
    self.output = nn.LazyLinear(10)

  def forward(self, X):
    return self.output(F.relu(self.hidden(X)))

In [24]:
net = MLP()

X = torch.randn(size=(2, 20))
Y = net(X)

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [25]:
torch.save(net.state_dict(), 'mlp.params')

In [26]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()                                      # evaluation mode

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


MLP(
  (hidden): LazyLinear(in_features=0, out_features=256, bias=True)
  (output): LazyLinear(in_features=0, out_features=10, bias=True)
)

In [28]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])